In [ ]:
#include <stdint.h>
#include <string.h>

uint16_t calculate_ip_checksum(const uint8_t* header, size_t length) {
    uint32_t sum = 0;
    const uint16_t* ptr = (const uint16_t*)header;
    
    // Sum all 16-bit words
    while (length > 1) {
        sum += *ptr++;
        length -= 2;
    }
    
    // Handle odd byte
    if (length > 0) {
        sum += *(uint8_t*)ptr << 8;
    }
    
    // Add carry bits
    while (sum >> 16) {
        sum = (sum & 0xFFFF) + (sum >> 16);
    }
    
    return ~sum & 0xFFFF;
}

// Alternative with manual byte handling for better portability
uint16_t calculate_ip_checksum_portable(const uint8_t* header, size_t length) {
    uint32_t sum = 0;
    
    // Sum 16-bit words
    for (size_t i = 0; i < length - 1; i += 2) {
        sum += (header[i] << 8) + header[i + 1];
    }
    
    // Handle odd byte
    if (length % 2) {
        sum += header[length - 1] << 8;
    }
    
    // Fold carry bits
    while (sum >> 16) {
        sum = (sum & 0xFFFF) + (sum >> 16);
    }
    
    return ~sum & 0xFFFF;
}

with above code, it assmes the checksum field is already reset to 0

## Checking if a Class Has a Function with Given Signature

There are several ways to check if a class has a function with a specific signature in C++:

In [ ]:
#include <iostream>
#include <type_traits>

// 1. SFINAE (Substitution Failure Is Not An Error) approach
template<typename T, typename = void>
struct has_foo : std::false_type {};

template<typename T>
struct has_foo<T, std::void_t<decltype(std::declval<T>().foo())>> : std::true_type {};

// More specific signature check
template<typename T, typename = void>
struct has_foo_int : std::false_type {};

template<typename T>
struct has_foo_int<T, std::void_t<decltype(std::declval<T>().foo(std::declval<int>()))>> : std::true_type {};

// Check for specific return type and parameters
template<typename T, typename = void>
struct has_bar_void_int : std::false_type {};

template<typename T>
struct has_bar_void_int<T, std::enable_if_t<
    std::is_same_v<void, decltype(std::declval<T>().bar(std::declval<int>()))>
>> : std::true_type {};

// Test classes
class ClassA {
public:
    void foo() {}
    void foo(int x) {}
    void bar(int x) {}
};

class ClassB {
public:
    int foo() { return 0; }
    std::string bar(int x) { return ""; }
};

class ClassC {
public:
    void baz() {}
};

int main() {
    std::cout << "ClassA has foo(): " << has_foo<ClassA>::value << std::endl;
    std::cout << "ClassA has foo(int): " << has_foo_int<ClassA>::value << std::endl;
    std::cout << "ClassA has bar(int) -> void: " << has_bar_void_int<ClassA>::value << std::endl;
    
    std::cout << "ClassB has foo(): " << has_foo<ClassB>::value << std::endl;
    std::cout << "ClassB has foo(int): " << has_foo_int<ClassB>::value << std::endl;
    std::cout << "ClassB has bar(int) -> void: " << has_bar_void_int<ClassB>::value << std::endl;
    
    std::cout << "ClassC has foo(): " << has_foo<ClassC>::value << std::endl;
    std::cout << "ClassC has foo(int): " << has_foo_int<ClassC>::value << std::endl;
    
    return 0;
}

In [ ]:
// 2. C++20 Concepts approach (cleaner syntax)
#include <concepts>

template<typename T>
concept HasFoo = requires(T t) {
    t.foo();
};

template<typename T>
concept HasFooInt = requires(T t, int x) {
    t.foo(x);
};

template<typename T>
concept HasBarVoidInt = requires(T t, int x) {
    { t.bar(x) } -> std::same_as<void>;
};

// Usage with concepts
template<HasFoo T>
void call_foo(T& obj) {
    obj.foo();
}

template<HasFooInt T>
void call_foo_int(T& obj, int x) {
    obj.foo(x);
}

// Example usage
void test_concepts() {
    ClassA a;
    ClassB b;
    
    if constexpr (HasFoo<ClassA>) {
        call_foo(a);
    }
    
    if constexpr (HasFooInt<ClassA>) {
        call_foo_int(a, 42);
    }
    
    // This would compile-time error if ClassC doesn't have foo()
    // call_foo(ClassC{});
}

In [ ]:
// 3. Macro-based approach for convenience
#define HAS_MEMBER_FUNCTION(func_name, signature) \
template<typename T, typename = void> \
struct has_##func_name : std::false_type {}; \
template<typename T> \
struct has_##func_name<T, std::void_t<decltype(std::declval<T>().func_name signature)>> : std::true_type {};

// Usage:
HAS_MEMBER_FUNCTION(process, (std::declval<int>(), std::declval<std::string>()))
HAS_MEMBER_FUNCTION(getValue, ())

// 4. Using std::is_invocable (C++17)
template<typename T, typename... Args>
constexpr bool is_callable_v = std::is_invocable_v<decltype(&T::foo), T*, Args...>;

// 5. Runtime approach using function pointers (for polymorphic cases)
template<typename T>
bool has_foo_runtime(T* obj) {
    // This approach works for virtual functions or when you need runtime checking
    try {
        auto ptr = &T::foo;  // Will fail to compile if foo doesn't exist
        return ptr != nullptr;
    } catch (...) {
        return false;
    }
}

// Example with more complex signature checking
template<typename T>
struct function_signature_checker {
    // Check if T has a method with signature: int method(const std::string&, double)
    template<typename U>
    static auto test_method(int) -> decltype(
        std::declval<U>().method(std::declval<const std::string&>(), std::declval<double>()),
        std::true_type{}
    );
    
    template<typename>
    static std::false_type test_method(...);
    
    static constexpr bool has_method = decltype(test_method<T>(0))::value;
};

## Summary of Approaches

1. **SFINAE with `std::void_t`** (C++17+): Most flexible, works with any compiler supporting C++17
2. **C++20 Concepts**: Cleanest syntax, but requires C++20
3. **Macro-based**: Convenient for repetitive checks, but can be less readable
4. **`std::is_invocable`** (C++17): Good for checking if a specific call is valid
5. **Runtime checking**: Useful for polymorphic scenarios

### When to use each:
- **Compile-time template selection**: Use SFINAE or Concepts
- **Template constraints**: Use Concepts (C++20) or SFINAE
- **Multiple similar checks**: Use macros
- **Polymorphic/runtime scenarios**: Use runtime approaches

### Best Practices:
- Prefer concepts when using C++20
- Use `std::void_t` with SFINAE for C++17
- Always use `std::declval<T>()` for perfect forwarding in unevaluated contexts
- Consider both function existence AND signature compatibility